In [1]:
import pandas as pd
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score,precision_score,f1_score


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [9]:

df = pd.read_csv('Sentiment_Dataset.csv')
df

,Sentence,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2743,I just got bored watching Jessice Lange take h...,0
2744,"Unfortunately, any virtue in this film's produ...",0
2745,"In a word, it is embarrassing.",0
2746,Exceptionally bad!,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   2748 non-null   object
 1   Sentiment  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [12]:
stop_words = set(nltk.corpus.stopwords.words('english'))
class CustomTokenizer:
    def tokenize(self, text):
        tokens = nltk.word_tokenize(text)
        lowercased_tokens = [token.lower() for token in tokens]
        filtered_tokens = [token for token in lowercased_tokens if token not in stop_words]
        return filtered_tokens



In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   2748 non-null   object
 1   Sentiment  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [14]:
df['Sentence']=df['Sentence'].astype('string')

In [15]:
df.head()

,Sentence,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [16]:
# Create an instance of the custom tokenizer
tokenizer = CustomTokenizer()

# Tokenize and preprocess the text column
processed_text = []
for text,tx in df.iterrows():
  ok=str(tx.Sentence)
  processed_text.append(' '.join(tokenizer.tokenize(ok)))

In [17]:
dfc=df.copy()

In [18]:
dfc['Sentence']=processed_text

In [19]:
dfc.head()

,Sentence,Sentiment
0,wow ... loved place .,1
1,crust good .,0
2,tasty texture nasty .,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices .,1


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Aplicar el tokenizador a la columna 'Sentence'
X_processed = [' '.join(tokenizer.tokenize(text)) for text in df['Sentence']]

# Codificación de etiquetas
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df['Sentiment'])

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.2, random_state=42)

print(f"Training size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Training size: 2198
Test size: 550


In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Definir el tokenizador
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")  
tokenizer.fit_on_texts(X_train) 

# Convertir texto a secuencias de números
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Hacer padding a las secuencias
maxlen = 100  
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Definir el modelo
model = Sequential()

# Capa de Embedding
model.add(Embedding(input_dim=10000,  
                    output_dim=128,   
                    input_length=100)) 

# Capa LSTM
model.add(LSTM(64, return_sequences=True))  
model.add(Dropout(0.2))  

# Segunda capa LSTM 
model.add(LSTM(32))  
model.add(Dropout(0.2))  

# Capa Densa de Salida
model.add(Dense(1, activation='sigmoid'))  

# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',  
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 lstm_4 (LSTM)               (None, 100, 64)           49408     
                                                                 
 dropout_4 (Dropout)         (None, 100, 64)           0         
                                                                 
 lstm_5 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1341857 (5.12 MB)
Trainable params: 1341